In [1]:
import pandas as pd
import numpy as np

# What we know:

#### The kaggle provided dataset contains 3 files:  
##### comment_to_score.csv
    - Contains all unranked comments. Our objective is to rank these comments based off the toxicity, with 1 being the most toxic. 

##### sample_submission.csv
    - Contains an example of how the submission should look like... Not sure if this will help with training?

##### validation_data.csv
    - According to kaggle this is the "pair rankings that can be used to validate models. Includes annotator work id and how the annotator ranked a given pair of comments"
    
# Understanding the objective
Before moving on, let's take a moment and understand what our overall objective is: 
We want to rank a list of comments base off of the level of toxicity without the use of any provided training data. However, while we do not have training data, we have provided validation data, which includes a set of toxicity rankings that might help to validate models.

# Understanding our data:
In the next few lines, we will import our data and see what we're actually dealing with and see if the data provided is actually useful. 

#### comments_to_score.csv
        - We knew how the data looked like before, but let's take a closer look and see what might need to be cleaned up  
   
   - Column Names
       - comment_id (comment identifier)
       - text (actual text)

In [2]:
# comments_to_score data
df = pd.read_csv("./data/comments_to_score.csv")
eg1 = df.text[0]
print(eg1)

"
 

Gjalexei, you asked about whether there is an ""anti-editorializing"" policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.

If something you write is edited and you're not sure why, please continue to question such edits on the talk page.  Sometimes, you'll learn more about wikipedia policy.  Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"


In [3]:
eg2 = df.text[25]
print(eg2)

"

I already gave you the source for all my edits   The Slur database, so I will not continue to play your little game. If you weren't so lazy and intent on harassment, you could use Google to search for ""Gargamel"" and ""Jew"". You get more than 400 hits including white supremacist sites and an academic paper dating to 1996. It's obviously a real slur with some usage. Nice try at being obdurate though. I'm sure there's a slur that describes that characteristic. Shyster doesn't quite cover it.  22:08, 22 Dec 2004 (UTC)"


##### took two comments from the provided dataset. We can make a few observations based off of the content of the texts:   
  
- First, punctuation. Normally, when we deal with datasets that involve texts, we would ignore punctuation. Howevever, knowing the habits of the internet, the punctuation can reflect toxicity. For instance, quotation marks have been used to represent sarcasm.
    -Building onto this, this same concept applies with capitalized letters. Generally speaking, when dealing with texts, we would try to make the letters consistent. However, given the habits of the internet, we can 
    
- I believe we can consider the second comment to be more toxic. But let's try to understand what makes it toxic: 
    - "If you weren't so lazy and intent on harassment, you could use Google to search for ""Gargamel"" and ""Jew""". 
        - This particular sentence feels targeted, with lazy being the negative description.   
        - Might want to consider pulling common words, phrases, and general patterns with the provided validation dataset as a means to start. However, keep in mind that the dataset also contains data that is not in our testing set

###### Validation_data.csv
After playing around with the data, I've come to realize the order of the comments do not matter. As described on Kaggle, this ia a collection of comments, which a person judged a pair of comments to see which one is more or less toxic (making almost binary). The comments are kinda hilarious from a third person perspective.  
  
Looking at the data on excel, I noticed that some comments are repeated, showing that it's been consistently rated as being more toxic than the other. It might be interesting to make a histogram based off of the number times a comment is rated as toxic. We can also create a list of toxic comments and have it ordered from most toxic to least toxic based off the number of counts it was rated as more toxic. 

In [4]:
vd = pd.read_csv("./data/validation_data.csv")
eg3 = vd.loc[0,:]
eg3

worker                                                      313
less_toxic              This article sucks \n\nwoo woo wooooooo
more_toxic    WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
Name: 0, dtype: object

In [5]:
eg4 = vd.loc[100,:]
eg4

worker                                                       46
less_toxic                           I'm Jim,a retarded idiot .
more_toxic     You are a nazi. \n\nYour defense of the Latin...
Name: 100, dtype: object

##### sample_submission.csv
This simply tells us how we should make our submission, it is interesting that we only care about the comment_id and the score. Although I am more interested in how the score will be developed. Is it unique or what?

In [6]:
eg5 = pd.read_csv("./data/sample_submission.csv")
eg5.loc[0:5,:]

,comment_id,score
0,114890,0.5
1,732895,0.5
2,1139051,0.5
3,1434512,0.5
4,2084821,0.5
5,2452675,0.5


### Testing out Ideas

In the previous section, I discussed ideas to explore. In this section, we will try out those ideas and see how they could apply to our model. 

In [7]:
## Find all the unique comments in the validation dataset. 

# Alright, so I want to look at this data with ease. To save time and memory, I'm going to give each unique comment a unique number
# And count the number of occurences the unique ID has. 
toxic_unique = vd.more_toxic.unique()
toxic_dict = {};
not_toxic_unique = vd.less_toxic.unique()
not_toxic_dict = {};
count = 0;

for tword in toxic_unique:
    toxic_dict[count] = [tword, len(vd.index[vd.more_toxic == tword].tolist())]
    count += 1
    # Create a dictionary that we can use
    #I don't know how much time or memory this uses, but basicaly I am creating a dictionary which stores the number of counts per unique comment

count = 0
for ntword in not_toxic_unique:
    not_toxic_dict[count] = [ntword, len(vd.index[vd.less_toxic == ntword].tolist())] # Create a dictionary that we can use
    count +=1

In [8]:
toxic_df = pd.DataFrame.from_dict(toxic_dict, orient = 'index', columns = ['text', 'count']).sort_values(by =['count'], ascending = False)
not_toxic_df = pd.DataFrame.from_dict(not_toxic_dict, orient = 'index', columns = ['text', 'count']).sort_values(by = ['count'], ascending = False)

toxic_df[0:20]

,text,count
3603,YOU SUCK \n\nI HATE YOU> YOU CAN FUCK MY ASS ...,14
905,STUPID SPANISH CENTRALISTSTUPID SPANISH CENTRA...,14
788,MORON! The USA has only 5% of the worlds popul...,12
1247,"LAST WARNING YOU FUCKING TOSSER, IT TAKES ME 1...",12
1990,Imma cut off ur nutz and send em to ur gf \n\...,12
1516,PEOPLE ON THIS SITE SUCK \n\nPEOPLE ON THIS ...,12
4523,"""\n\nBastard Child\nDoes anyone know if Obama ...",12
2130,INTEVIEW \n\nME: ARE YOU GAY?\n\nABE GAY DASH...,12
563,YOU'RE A FUCKING LIAR WHO DOESN'T KNOW HOW TO ...,12
695,ScJessey is a Big Fat Faggot Pinko Commie Asshole,12


In [9]:
not_toxic_df[0:20]

,text,count
5422,How many sockpuppets do you have?,13
786,sorry i jumped to conclusions \n\non christia...,13
2192,"Sockpuppet\nHi, would you please confirm this ...",12
4930,I erased your cuss word\nFrom: some random per...,12
4021,An addition comment of interest by Andrew Coyn...,12
6612,this irishtom guy is turning every article int...,12
4646,Please no Wiki-Weirdo posts !,11
9383,the fife shuffle \n\naahh dismas surely you a...,11
1401,"""\n\n Please stop your disruptive editing. If ...",11
4043,You are not sorry one damned bit. You have ye...,11


In [10]:
print("toxic_df length: ", len(toxic_df))
print("not_toxic_df length: ", len(not_toxic_df))

toxic_df length:  11678
not_toxic_df length:  11532


As you can see, I created two different dataframes to rank the frequency of comments found in the toxic and less toxic columns from the validity data provided. One thing to keep in mind is that we will want our algorithm to generalize, therefore having a keyword classifer might not be particularly useful. Instead of looking for specific words, we should think about the underlying trends in the data.      
  
Interestingly, in the toxic column, a lot of the comments use an abundance of capital letters. Furthermore, comments deemed more toxic seem to have more misspelling, slang, and punctuation. Of course, this type of identification is for the most obvious cases. However, this leads me to wonder if we can apply an algorithm to identify underlying trends.   
  
I almost glossed over one thing: The goal of this project is to rank the comments from most toxic to least toxic. As explained in the project description, it is very difficult to rank a list of comments entirely. Instead, it is a lot easier to judge two comments and rank which one is toxic. I should keep this in mind when I'm building my future algorithm. Instead of loading the entire dataset into my model, it might be better to use a sliding window method that takes in two comments and spits out which one is considered more toxic. This leads me to think that a binary classifier might be particularly useful. However, how would we identify the unlabeled features? Hmmm.... How much natural language processing is actually necessary here?
  
On a side note, I think it might be interesting to see the amount of overlap between the two classes (more toxic vs less toxic)
I find it funny that there's more toxic comments than not toxic comments. 

In [11]:
common_df = {}
un_not_toxic_df = {}


count = 0
ncount = 0

for i in range(len(not_toxic_df.text)):
    ntword = not_toxic_df.text[i]
    ntcount = not_toxic_df['count'][i]

    if ntword in list(toxic_df.text):
        t_idx = (toxic_df.index[toxic_df.text == ntword].tolist())[0]
        t_count = toxic_df['count'][t_idx]
        tword = toxic_df['text'][t_idx]
        common_df[count] = [ntword, ntcount, t_count]
        count += 1
        
    else:
        un_not_toxic_df[ncount] = [ntword, ntcount]
        ncount += 1
        
common_df = pd.DataFrame.from_dict(common_df, orient = 'index', columns = ['text', 'ntcount', 'tcount'])
un_not_toxic_df = pd.DataFrame.from_dict(un_not_toxic_df, orient = 'index', columns = ['text', 'count']).sort_values(by = ['count'], ascending = False)
un_not_toxic_df = un_not_toxic_df.reset_index(drop = True)

In [12]:
count = 0
un_toxic_df = {}

for i in range(len(toxic_df)):
    tword = toxic_df.text[i]
    if tword not in list(common_df.text):
        tcount = toxic_df['count'][i]
        un_toxic_df[count] = [tword, tcount]
        count += 1

un_toxic_df = pd.DataFrame.from_dict(un_toxic_df, orient = 'index', columns = ['text', 'count']).sort_values(by = ['count'], ascending = False)
un_toxic_df = un_toxic_df.reset_index(drop = True)

In [13]:
common_df = common_df.sort_values(by = ['tcount', 'ntcount'], ascending = False)
print(common_df[0:10])
print("\nshared comments count: ", len(common_df))
print("toxic comments total: ", len(toxic_df))
print("less toxic comments total: ", len(not_toxic_df))
print("% of overlap for toxic: ", (round(len(common_df)/len(toxic_df),3)*100))
print("% of overlap for less toxic: ", (round(len(common_df)/len(not_toxic_df),3)*100))

                                                   text  ntcount  tcount
302   STUPID SPANISH CENTRALISTSTUPID SPANISH CENTRA...        1      14
7223   YOU SUCK \n\nI HATE YOU> YOU CAN FUCK MY ASS ...        1      14
299   ScJessey is a Big Fat Faggot Pinko Commie Asshole        3      12
6370   Imma cut off ur nutz and send em to ur gf \n\...        3      12
759   "\nNo, I'm not joking.  I'm hoping to help BD ...        4      11
1116  Fuck you, Smith.  Please have me notified when...        1      11
2973   Gonna fuck you up \n\nYou little bitch. 86.17...        1      11
3768  Missrain4, I will make a rain of cum on your f...        1      11
4623  MOTHERFUCKER, DONT BE CHANGING THE FUCKING GTA...        1      11
4791                      |decline=Fucking bellend —           1      11

shared comments count:  8959
toxic comments total:  11678
less toxic comments total:  11532
% of overlap for toxic:  76.7
% of overlap for less toxic:  77.7


This is interesting... There is a huge overlap between our more toxic and less toxic comments. Furthermore, the percentage of overlap is similar in both cases, with 76.7% overlap for toxic and 77.7% for less toxic which makese sense given that both datasets are similar sizes. This also tells me that close to 25% of each dataset is uniquely toxic and not toxic, which is also worth exploring

In [14]:
print("Uniquely Toxic Comments, length: ", len(un_toxic_df))
un_toxic_df[0:10]

Uniquely Toxic Comments, length:  2719


,text,count
0,go to hell \n\ngo to hell and leave me alone,12
1,"LAST WARNING YOU FUCKING TOSSER, IT TAKES ME 1...",12
2,"""\n\nBastard Child\nDoes anyone know if Obama ...",12
3,YOU'RE A FUCKING LIAR WHO DOESN'T KNOW HOW TO ...,12
4,INTEVIEW \n\nME: ARE YOU GAY?\n\nABE GAY DASH...,12
5,MORON! The USA has only 5% of the worlds popul...,12
6,PEOPLE ON THIS SITE SUCK \n\nPEOPLE ON THIS ...,12
7,suck my dick irish guy you faggot,9
8,YOU ARE A FUCKING NERD. GO KILL YOURSELF YOU N...,9
9,"You dummasses, I DID SAY THE DAMN RIGHT STUFF...",9


In [15]:
print("Uniquely Not Toxic Comments, length: ", len(un_not_toxic_df))
un_not_toxic_df[0:10]

Uniquely Not Toxic Comments, length:  2573


,text,count
0,An addition comment of interest by Andrew Coyn...,12
1,"Sockpuppet\nHi, would you please confirm this ...",12
2,", anything else inserted here will be removed",9
3,ru-sib \n\nThe number of bots increases. You ...,9
4,""".\n\n""""This emphasis also led Boas to conclud...",9
5,"""\n\n The above argument \n\nI just discerned ...",9
6,"""\nSahih Hadith are only second in authority a...",9
7,How is that vandalism? \n\nI'm the most recen...,9
8,"BTW \n\nI've noticed your page, and with all ...",9
9,If you want to post rubbish about Meyer Moran ...,9


I believe looking at this new data supports what I was saying previously, which is that all caps comments are a great way to identify toxic comments. However, as I said before, this will only help to discern the most obvious cases. This remains true with regarding less toxic comments and proper grammar. Later, as I build my model, I should keep in mind ways to check for proper spelling and grammar. 

If I think about this in larger scenario, should I create my own scoring method. This might help with unsupervised learning. For example, I could add points per word that is in all caps, which w